<a href="https://colab.research.google.com/github/simulate111/Textual-Data-Analysis-25/blob/main/ner_inference_ex5(solution).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==1

In [2]:
# Download data
!wget http://dl.turkunlp.org/TKO_8964_2023/news-fi-2019.jsonl
!wget http://dl.turkunlp.org/TKO_8964_2023/news-fi-2021.jsonl
!wget http://dl.turkunlp.org/TKO_8964_2023/news-en-2019.jsonl
!wget http://dl.turkunlp.org/TKO_8964_2023/news-en-2021.jsonl

--2025-02-05 08:40:30--  http://dl.turkunlp.org/TKO_8964_2023/news-fi-2019.jsonl
Resolving dl.turkunlp.org (dl.turkunlp.org)... 195.148.30.23
Connecting to dl.turkunlp.org (dl.turkunlp.org)|195.148.30.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97242739 (93M) [application/octet-stream]
Saving to: ‘news-fi-2019.jsonl’

news-fi-2019.jsonl  100%[===================>]  92.74M  18.6MB/s    in 6.4s    

2025-02-05 08:40:36 (14.6 MB/s) - ‘news-fi-2019.jsonl’ saved [97242739/97242739]

--2025-02-05 08:40:36--  http://dl.turkunlp.org/TKO_8964_2023/news-fi-2021.jsonl
Resolving dl.turkunlp.org (dl.turkunlp.org)... 195.148.30.23
Connecting to dl.turkunlp.org (dl.turkunlp.org)|195.148.30.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36139303 (34M) [application/octet-stream]
Saving to: ‘news-fi-2021.jsonl’

news-fi-2021.jsonl  100%[===================>]  34.46M  10.0MB/s    in 3.4s    

2025-02-05 08:40:40 (10.0 MB/s) - ‘news-fi-2021.json

In [3]:
# create a dataset
from datasets import load_dataset

dataset_fi = load_dataset('json', data_files=['news-fi-2021.jsonl']) # Finnish is bigger, so one file is enough
dataset_en = load_dataset('json', data_files=['news-en-2019.jsonl', 'news-en-2021.jsonl'])
print("Finnish:", dataset_fi)
print("English:", dataset_en)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Finnish: DatasetDict({
    train: Dataset({
        features: ['summary', 'tags', 'text', 'timestamp', 'title', 'url'],
        num_rows: 7482
    })
})
English: DatasetDict({
    train: Dataset({
        features: ['summary', 'tags', 'text', 'timestamp', 'title', 'url'],
        num_rows: 3540
    })
})


# Models

## [Kansallisarkisto/finbert-ner](https://huggingface.co/Kansallisarkisto/finbert-ner)

* Fine-tuned TurkuNLP/bert-base-finnish-cased-v1
* 10 named entity categories
  * PERSON
  * ORG
  * LOC
  * GPE
  * PRODUCT
  * EVENT
  * DATE
  * JON (Finnish journal numbers / diaarinumero)
  * FIBC (Finnish business identity codes / y-tunnus)
  * NORP (nationality, religious and political groups)
* Training data contains:
  * Turku OntoNotes Entities Corpus
  * Finnish part of the NewsEye dataset (historical newspapers)
  * Several other sources, including administrative documents, old books, and thesis works.

## [dslim/bert-base-NER](https://huggingface.co/dslim/bert-base-NER)

* Fine-tuned bert-base-cased
* 4 named entity categories
  * LOC
  * ORG
  * PER
  * MISC
* Trained on CoNLL2003 Englihs data




In [4]:
import transformers

model_fi='Kansallisarkisto/finbert-ner'
pipeline_fi = transformers.pipeline('token-classification', model=model_fi, tokenizer=model_fi)
model_en='dslim/bert-base-NER'
pipeline_en = transformers.pipeline('token-classification', model=model_en, tokenizer=model_en)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/424k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
def validate(previous, current):
  # Beginning tag and outside tag are always valid (if we do not restric entities to token level)
  if current["entity"].startswith("B-") or current["entity"]=="O":
    return "valid transition"
  if current["entity"].startswith("I-"):
    # Inside is not valid if (1) first token of a document, (2) previous token is outside, or (3) entity type does not match
    if current["start"] == 0:
      return "first token of a document cannot be inside"
    if previous["entity"] == "O":
      return "O --> I"
    if previous["entity"].split("-")[1] != current["entity"].split("-")[1]:
      return f"{previous['entity']} --> {current['entity']}"
    return "valid transition"
  assert "Should not happen!"

In [6]:
from itertools import pairwise
from collections import Counter
from tqdm import tqdm

# count invalid label transitions
print("\nFinnish:")
c = Counter()
outputs = pipeline_fi([s["text"] for s in dataset_fi["train"]], ignore_labels = [])
for document_predictions in outputs:
  for previous_tag, current_tag in pairwise(document_predictions):
    valid_type = validate(previous_tag, current_tag)
    c.update([valid_type])
for valid_type, count in c.most_common(10):
  print(valid_type, count)


print("\nEnglish:")
c = Counter()
outputs = pipeline_en([s["text"] for s in dataset_en["train"]], ignore_labels = [])
for i, document_predictions in enumerate(outputs):
  for previous_tag, current_tag in pairwise(document_predictions):
    valid_type = validate(previous_tag, current_tag)
    c.update([valid_type])
for valid_type, count in c.most_common(10):
  print(valid_type, count)


Finnish:
valid transition 3171523
O --> I 3760
B-GPE --> I-ORG 197
B-ORG --> I-PRODUCT 118
I-ORG --> I-PRODUCT 117
B-LOC --> I-GPE 111
B-PRODUCT --> I-ORG 99
I-GPE --> I-ORG 84
I-LOC --> I-GPE 70
I-PRODUCT --> I-ORG 66

English:
valid transition 1461764
O --> I 3015
B-MISC --> I-ORG 288
I-PER --> I-ORG 271
B-PER --> I-ORG 256
I-LOC --> I-ORG 224
I-ORG --> I-LOC 214
I-ORG --> I-MISC 166
B-ORG --> I-LOC 143
B-ORG --> I-MISC 132


In [7]:
# let's add aggregation_strategy from here: https://github.com/huggingface/transformers/blob/b1954fd64abf392a60e3e007f03471db0f3cf4db/src/transformers/pipelines/token_classification.py#L76
outputs = pipeline_en([s["text"] for s in dataset_en["train"]], aggregation_strategy="max")


In [8]:
type_counter = Counter()
entity_counter = Counter()

for doc_output in outputs:
  for ne in doc_output: # aggregated ne
    type_counter.update([ne["entity_group"]])
    entity_counter.update([(ne["word"], ne["entity_group"])])
print("Most common types:")
for type, count in type_counter.most_common():
  print(type, count)
print("\nMost common entities:")
for entity, count in entity_counter.most_common(20):
  print(entity, count)

Most common types:
ORG 23970
LOC 20837
PER 13263
MISC 10449

Most common entities:
('Finland', 'LOC') 7627
('Finnish', 'MISC') 3351
('Helsinki', 'LOC') 1698
('EU', 'ORG') 1018
('Yle', 'ORG') 875
('Covid', 'MISC') 752
('THL', 'ORG') 463
('SDP', 'ORG') 452
('Finns Party', 'ORG') 427
('Sweden', 'LOC') 375
('Helsingin Sanomat', 'ORG') 375
('Swedish', 'MISC') 372
('HS', 'ORG') 370
('Russia', 'LOC') 341
('Centre Party', 'ORG') 322
('Russian', 'MISC') 318
('Marin', 'PER') 307
('Tampere', 'LOC') 289
('European', 'MISC') 288
('Niinistö', 'PER') 280
